In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init
import time
import random
import pandas as pd
import openpyxl

In [ ]:
#### material params:- RT-35 ####
k_therm = 0.2
L = 160000
Cp = 2000
rho = 800 
T_solidus = 305
T_liquidus = 313

#### Heat flux ####
q = 225

#### Normalising coeffs ####
T_change = 0.35
delta_x = 0.02
delta_y = 0.02
delta_T = 17.14
delta_t = 1200

#### Normalised Coefficients  ####
k1 = k_therm/(rho*Cp*delta_x**2)*delta_t
k2 = k_therm/(rho*(Cp+L/(T_liquidus - T_solidus))*delta_x**2)*delta_t
q_norm = q*delta_x/(k_therm*delta_T)

#### FVM Domain Parameters ####
N_x = 81
N_y = 81
x_l = 0
y_b = 0
x_r = 1
y_t = 1
N_t = 250000
dt = 0.00001

print('k1 = ',k1,' k2 = ', k2, ' q_norm = ', q_norm)

In [ ]:
# X, Y matrix
X = np.linspace(x_l,x_r,N_x)
dx = X[1] - X[0]
X = np.tile(X,N_y)
X = np.reshape(X, (N_x,N_y))

Y = np.linspace(y_b,y_t,N_y)
dy = Y[1] - Y[0]
Y = np.repeat(Y,N_x)
Y = np.reshape(Y, (N_x,N_y))

# liquid fraction, battery and initial temperature matrix
f = np.zeros((N_x,N_y))
T = np.zeros((N_x,N_y))

In [16]:
aN = np.zeros((N_x,N_y))
aS = np.zeros((N_x,N_y))
aE = np.zeros((N_x,N_y))
aW = np.zeros((N_x,N_y))
aP = np.zeros((N_x,N_y))
flag = np.zeros((N_x,N_y))

for i in range(N_x):
    for j in range(N_y):
        
        if Y[i][j]!=y_t:
            aN[i][j] = k2*dt/dy**2
            
        if Y[i][j]!=y_b:
            aS[i][j] = k2*dt/dy**2
        
        if X[i][j]!=x_l:
            aW[i][j] = k2*dt/dx**2
            
        if X[i][j]!=x_r:
            aE[i][j] = k2*dt/dx**2
            
        aP[i][j] = 1 - (aN[i][j] + aS[i][j] + aW[i][j] + aE[i][j])

In [ ]:
storage_interval = 10000

T_str = []
T_str.append(np.copy(T))
T_new = np.copy(T)

f_str = []
f_str.append(np.copy(f))
f_new = np.copy(f)      

for t in range(N_t):
    if t%10000==0:
        print('t = ',(t+1)*dt)
    
    T = np.copy(T_new)
    f = np.copy(f_new)

    # Temperature Update
    for i in range(N_y):
        for j in range(N_x):

            if ( X[i][j]!=x_l and X[i][j]!=x_r and Y[i][j]!=y_t and Y[i][j]!=y_b ):
                
                T_new[i][j] = ( aN[i][j]*T[i+1][j] + aS[i][j]*T[i-1][j] + 
                                   aW[i][j]*T[i][j-1] + aE[i][j]*T[i][j+1] + aP[i][j]*T[i][j] )
                
                f_new[i][j] = T_new[i][j]/T_change

    # LEFT BOUNDARY
    for i in range(1, N_y - 1):
        T_new[i][0] = q_norm*dx + T_new[i][1]
        f_new[i][0] = T_new[i][0]/T_change

    # BOTTOM BOUNDARY
    for j in range(1, N_x - 1):
        T_new[0][j] = q_norm*dy + T_new[1][j]
        f_new[0][j] = T_new[0][j]/T_change

    # TOP BOUNDARY
    for i in [N_y-1]:
        for j in range(N_x):
            T_new[i][j] = T_new[i-1][j]
            f_new[i][j] = T_new[i][j]/T_change
    # RIGHT BOUNDARY
    for j in [N_x-1]:
        for i in range(N_y):
            T_new[i][j] = T_new[i][j-1]
            f_new[i][j] = T_new[i][j]/T_change

    # CORNER POINT
    T_new[0][0] = q_norm*dx + T_new[1][0] 
    f_new[0][0] = T_new[0][0]/T_change

    ######### Liquid Fraction Update #########
    for i in range(N_y):
        for j in range(N_x):
        
            if T_new[i][j]>=T_change or f_new[i][j]>=1:

                f_new[i][j] = 1

                if Y[i][j]!=y_t:
                    aN[i][j] = k1*dt/dy**2

                if Y[i][j]!=y_b:
                    aS[i][j] = k1*dt/dy**2

                if X[i][j]!=x_l:
                    aW[i][j] = k1*dt/dx**2

                if X[i][j]!=x_r:
                    aE[i][j] = k1*dt/dx**2

                aP[i][j] = 1 - (aN[i][j] + aS[i][j] + aW[i][j] + aE[i][j])

    if (t+1)%storage_interval==0:
        T_str.append(np.copy(T_new))
        f_str.append(np.copy(f_new))